<a href="https://colab.research.google.com/github/NBK-code/TPS_Gradient_Matching/blob/main/Experiments_MB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Import Libraries

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import torch.nn.functional as F

In [ ]:
#install opencv
!pip install opencv-python

In [ ]:
import cv2
import os

#Cholesky Decomposition and Reparameterization for Multivariate Gaussian

In [ ]:
# Define a random lower triangular matrix L and center c
np.random.seed(42)  # For reproducibility

L = np.array([[2.0, 0.0], [-2.0, 2.0]])  # Random lower triangular matrix
c = np.array([1.0, 2.0])  # Center of the potential

# Generate samples
num_samples = 100
epsilon = np.random.randn(num_samples, 2)  # Standard normal distribution (500, 2)
x_samples = c + epsilon @ L.T  # Generate samples using x = c + L * epsilon


# Define the potential function V(x, y)
def V(x, y, L, c):
    diff = np.array([x - c[0], y - c[1]])
    return 0.5 * diff.T @ np.linalg.inv( L @ L.T) @ diff

# Create a grid for plotting
x_grid, y_grid = np.meshgrid(np.linspace(-25, 25, 100), np.linspace(-25, 25, 100))
V_grid = np.zeros_like(x_grid)

# Calculate the potential at each grid point
for i in range(x_grid.shape[0]):
    for j in range(x_grid.shape[1]):
        V_grid[i, j] = V(x_grid[i, j], y_grid[i, j], L, c)

# Plot the potential and the samples
plt.figure(figsize=(8, 6))
plt.contourf(x_grid, y_grid, V_grid, levels=30, cmap='Blues')
plt.colorbar(label='Potential V(x, y)')
plt.scatter(x_samples[:, 0], x_samples[:, 1], color='red', s=10, label='Samples')
plt.scatter(c[0], c[1], color='black', marker='x', s=100, label='Center c')
plt.title('Potential V(x, y) and Random Samples')
plt.xlabel('x')
plt.ylabel('y')
plt.legend()
plt.show()

#Muller-Brown Potential

In [ ]:
def U(xs, beta=1.0):
    x, y = xs[:, 0], xs[:, 1]
    e1 = -200 * torch.exp(-(x - 1) ** 2 - 10 * y ** 2)
    e2 = -100 * torch.exp(-x ** 2 - 10 * (y - 0.5) ** 2)
    e3 = -170 * torch.exp(-6.5 * (0.5 + x) ** 2 + 11 * (x + 0.5) * (y - 1.5) - 6.5 * (y - 1.5) ** 2)
    e4 = 15.0 * torch.exp(0.7 * (1 + x) ** 2 + 0.6 * (x + 1) * (y - 1) + 0.7 * (y - 1) ** 2)
    return beta * (e1 + e2 + e3 + e4)

In [ ]:
#Plot potential U
A = np.array([[-0.55828035, 1.44169]])
B = np.array([[0.62361133, 0.02804632]])

x, y = np.linspace(-1.5, 1.0, 100), np.linspace(-0.5, 1.7, 100)
x, y = np.meshgrid(x, y)
xs = torch.tensor(np.stack([x, y], -1).reshape(-1, 2), dtype=torch.float32)
z = U(xs).reshape([100, 100])
z = z.detach().numpy()
plt.contourf(x, y, z, 50)
plt.scatter(A[0, 0], A[0, 1])
plt.scatter(B[0, 0], B[0, 1])
plt.show()

In [ ]:
# Gradient of U with respect to xs
def dUdx_fn(xs):
    xs.requires_grad_(True)  # Ensure gradients are tracked
    if xs.grad is not None:
        xs.grad = None  # Zero out any previous gradients

    U_val = U(xs).sum()  # Compute U and sum it (scalar result required for backward)
    U_val.backward()      # Compute the gradient of U with respect to xs
    return xs.grad        # Return the computed gradients

In [ ]:
# Analytical gradient computation of Muller-Brown potential
def grad_U_analytical(xs):
    x, y = xs[..., 0], xs[..., 1]

    # Compute each e_i for gradient computation
    e1 = -200 * torch.exp(-(x - 1) ** 2 - 10 * y ** 2)
    e2 = -100 * torch.exp(-x ** 2 - 10 * (y - 0.5) ** 2)
    e3 = -170 * torch.exp(-6.5 * (0.5 + x) ** 2 + 11 * (x + 0.5) * (y - 1.5) - 6.5 * (y - 1.5) ** 2)
    e4 = 15.0 * torch.exp(0.7 * (1 + x) ** 2 + 0.6 * (x + 1) * (y - 1) + 0.7 * (y - 1) ** 2)

    # Derivatives of e1 with respect to x and y
    dU1_dx = e1 * (-2 * (x - 1))
    dU1_dy = e1 * (-20 * y)

    # Derivatives of e2 with respect to x and y
    dU2_dx = e2 * (-2 * x)
    dU2_dy = e2 * (-20 * (y - 0.5))

    # Derivatives of e3 with respect to x and y
    dU3_dx = e3 * (-13 * (x + 0.5) + 11 * (y - 1.5))
    dU3_dy = e3 * (11 * (x + 0.5) - 13 * (y - 1.5))

    # Derivatives of e4 with respect to x and y
    dU4_dx = e4 * (1.4 * (x + 1) + 0.6 * (y - 1))
    dU4_dy = e4 * (0.6 * (x + 1) + 1.4 * (y - 1))

    # Sum the derivatives to get the total gradients with respect to x and y
    grad_U_x = dU1_dx + dU2_dx + dU3_dx + dU4_dx
    grad_U_y = dU1_dy + dU2_dy + dU3_dy + dU4_dy

    # Stack the gradients along the last axis to return a tensor of shape (N, 2)
    return torch.stack([grad_U_x, grad_U_y], dim=-1)

In [ ]:
# Generate meshgrid
x_vals, y_vals = np.linspace(-1.5, 1.0, 50), np.linspace(-0.5, 1.7, 50)
x_grid, y_grid = np.meshgrid(x_vals, y_vals)
xs = torch.tensor(np.stack([x_grid, y_grid], -1).reshape(-1, 2), dtype=torch.float32)

# Calculate potential values and gradients
U_vals = U(xs).detach().numpy().reshape(50, 50)  # Reshape to grid for plotting
grads = dUdx_fn(xs).detach().numpy().reshape(50, 50, 2)  # Gradients in x and y

grad_U_analytical_vals = grad_U_analytical(xs).detach().numpy().reshape(50, 50, 2)  # Analytical gradients

# Extract the components of the gradients
grad_x = grads[:, :, 0]  # Gradient w.r.t. x
grad_y = grads[:, :, 1]  # Gradient w.r.t. y

# Extract the components of the analytical gradients
grad_x_analytical = grad_U_analytical_vals[:, :, 0]
grad_y_analytical = grad_U_analytical_vals[:, :, 1]

# Plot the potential and the gradients as arrows
plt.figure(figsize=(10, 6))

# Plot the potential as a contour map
plt.contourf(x_grid, y_grid, U_vals, levels=20, cmap='viridis')
plt.colorbar(label='Potential U(x, y)')

# Plot the gradient vectors as arrows
plt.quiver(x_grid, y_grid, -grad_x, -grad_y, color='red')  # Arrows for gradients (flip direction for descent)

plt.title('Potential U(x, y) and Gradients')
plt.xlabel('x')
plt.ylabel('y')
plt.show()

# Plot the potential and the gradients as arrows
plt.figure(figsize=(10, 6))

# Plot the potential as a contour map
plt.contourf(x_grid, y_grid, U_vals, levels=20, cmap='viridis')
plt.colorbar(label='Potential U(x, y)')

# Plot the gradient vectors as arrows
plt.quiver(x_grid, y_grid, -grad_x_analytical, -grad_y_analytical, color='red')  # Arrows for gradients (flip direction for descent)

plt.title('Potential U(x, y) and Gradients')
plt.xlabel('x')
plt.ylabel('y')
plt.show()

In [ ]:
#Check if grad_x == grad_x_analytical within a tolerance
np.allclose(grad_x, grad_x_analytical, atol=1e-4)

#Mean Neural Network

In [ ]:
# Neural network NN_{\theta}(t) definition
class NN_theta(nn.Module):
    def __init__(self):
        super(NN_theta, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(1, 256),  # Input t (scalar)
            nn.ReLU(),
            nn.Linear(256, 256),
            nn.ReLU(),
            nn.Linear(256, 256),
            nn.ReLU(),
            nn.Linear(256, 2)  # Output shape should match A and B (2D vector here)
        )

    def forward(self, t):
        return self.fc(t)

# Instantiate the neural network
nn_theta = NN_theta()


def c_theta(t, A, B, nn_theta):
    # t should be of shape (N, 1) for a batch of N samples
    t = t.view(-1, 1)  # Ensure t is treated as a column vector
    return (1 - t) * A + t * B + t * (1 - t) * nn_theta(t)

In [ ]:
# Predefined A and B (assumed to be vectors here, adjust as needed)
A = torch.tensor([-0.55828035, 1.44169], dtype=torch.float32)
B = torch.tensor([0.62361133, 0.02804632], dtype=torch.float32)

# Evaluate c_theta(t) over a range of t
t_vals = torch.linspace(0, 1, 100)  # t from 0 to 1
c_t_vals = c_theta(t_vals, A, B, nn_theta).detach().numpy()

In [ ]:
#Plot potential U
A = np.array([[-0.55828035, 1.44169]])
B = np.array([[0.62361133, 0.02804632]])

x, y = np.linspace(-1.5, 1.0, 100), np.linspace(-0.5, 1.7, 100)
x, y = np.meshgrid(x, y)
xs = torch.tensor(np.stack([x, y], -1).reshape(-1, 2), dtype=torch.float32)
z = U(xs).reshape([100, 100])
z = z.detach().numpy()
plt.contourf(x, y, z, 50)
plt.scatter(A[0, 0], A[0, 1])
plt.scatter(B[0, 0], B[0, 1])
plt.plot(c_t_vals[:, 0], c_t_vals[:, 1])
plt.show()

#Generate Samples and Gradients of V

In [ ]:
# Constant lower triangular matrix L
L = torch.tensor([[0.2, 0.0], [0.0, 0.2]], dtype=torch.float64)

In [ ]:
# Scaled matrix L_scaled
def L_scaled(t, L):
    return torch.sqrt(t * (1 - t)).view(-1, 1, 1) * L  # Scales L by sqrt(t(1-t))

In [ ]:
def generate_samples_and_gradients_V(t, N, A, B, L, nn_theta):
    # Generate Gaussian noise
    epsilon = torch.randn(N, 2)

    # Compute c_theta(t)
    c_t = c_theta(t, A, B, nn_theta) # This should output a (2,) vector

    # Scale the matrix L
    L_scale = L_scaled(t, L) # This should output a (2, 2) matrix

    # Generate samples
    x_t = c_t + epsilon.double() @ L_scale.transpose(-1, -2)  # (N, 2)

    #Find inverse
    L_inv = torch.inverse(L_scale) # Should be a (2, 2) matrix

    #Find gradients
    grad_V = epsilon.double() @ L_inv # (N, 2)

    return x_t, grad_V

In [ ]:
# Generate meshgrid for Muller-Brown potential
x_vals, y_vals = np.linspace(-1.5, 1.0, 100), np.linspace(-0.5, 1.7, 100)
x_grid, y_grid = np.meshgrid(x_vals, y_vals)
xs = torch.tensor(np.stack([x_grid, y_grid], -1).reshape(-1, 2), dtype=torch.float32)

# Compute potential values
U_vals = U(xs).detach().numpy().reshape(100, 100)

# Sample t value and generate samples for t
t = torch.tensor([0.5], dtype=torch.float32)  # Example t value
N = 100  # Number of samples
samples, grad_V = generate_samples_and_gradients_V(t, N, A, B, L, nn_theta)  # Generate samples

# Convert samples to numpy arrays
samples = samples.detach().numpy()
grad_V = grad_V.detach().numpy()

# Plot the Muller-Brown potential and the generated samples
plt.figure(figsize=(8, 6))

# Plot the Muller-Brown potential as a contour map
plt.contourf(x_grid, y_grid, U_vals, levels=20, cmap='viridis')
plt.colorbar(label='Muller-Brown Potential U(x, y)')

# Overlay the generated samples
plt.scatter(samples[0,:,0], samples[0,:,1], color='red', marker='o', label=r'Samples $x_t$', s=0.8)
plt.quiver(samples[0,:,0], samples[0,:,1], -grad_V[0,:,0], -grad_V[0,:,1], color='orange', scale=700, label='Gradients')
plt.title(r'Muller-Brown Potential U(x, y) and Samples $x_t$')
plt.xlabel('x')
plt.ylabel('y')
plt.legend()
plt.show()

#Gradients of U computed at samples of V

In [ ]:
# Constant lower triangular matrix L
L = torch.tensor([[0.15, 0.0], [0.0, 0.15]], dtype=torch.float64)

In [ ]:
# Scaled matrix L_scaled
def L_scaled(t, L):
    return torch.sqrt(t * (1 - t)).view(-1, 1, 1) * L  # Scales L by sqrt(t(1-t))

In [ ]:
def generate_samples_and_gradients_V_U(t, N, A, B, L, nn_theta):
    # Generate Gaussian noise
    epsilon = torch.randn(N, 2)

    # Compute c_theta(t)
    c_t = c_theta(t, A, B, nn_theta)  # This should output a (2,) vector

    # Scale the matrix L
    L_scale = L_scaled(t, L)  # This should output a (2, 2) matrix

    # Generate samples: x_t = c_t + epsilon @ L_scale.T
    x_t = c_t + epsilon.double() @ L_scale.double().transpose(-1, -2)  # (N, 2)

    # Find inverse of L_scaled
    L_inv = torch.inverse(L_scale)  # Should be a (2, 2) matrix

    # Find gradients of V: grad_V = epsilon @ L_inv
    grad_V = epsilon.double() @ L_inv.double()  # (N, 2)

    # Compute the gradients of the Muller-Brown potential U at the samples x_t
    grad_U = grad_U_analytical(x_t)  # (N, 2), gradients of U at x_t

    return x_t, grad_V, grad_U

In [ ]:
# Generate meshgrid for Muller-Brown potential
x_vals, y_vals = np.linspace(-1.5, 1.0, 100), np.linspace(-0.5, 1.7, 100)
x_grid, y_grid = np.meshgrid(x_vals, y_vals)
xs = torch.tensor(np.stack([x_grid, y_grid], -1).reshape(-1, 2), dtype=torch.float32)

# Compute potential values
U_vals = U(xs).detach().numpy().reshape(100, 100)

# Sample t value and generate samples for t
t = torch.tensor([0.5], dtype=torch.float32)  # Example t value
N = 100  # Number of samples
samples, grad_V, grad_U = generate_samples_and_gradients_V_U(t, N, A, B, L, nn_theta)  # Generate samples

# Convert samples to numpy arrays
samples = samples.detach().numpy()
grad_V = grad_V.detach().numpy()
grad_U = grad_U.detach().numpy()

# Plot the Muller-Brown potential and the generated samples
plt.figure(figsize=(8, 6))

# Plot the Muller-Brown potential as a contour map
plt.contourf(x_grid, y_grid, U_vals, levels=20, cmap='viridis')
plt.colorbar(label='Muller-Brown Potential U(x, y)')

# Overlay the generated samples
plt.scatter(samples[0,:,0], samples[0,:,1], color='red', marker='o', label=r'Samples $x_t$', s=0.8)
plt.quiver(samples[0,:,0], samples[0,:,1], -grad_V[0,:,0], -grad_V[0,:,1], color='orange', scale=700, label='Gradients')
plt.title(r'Muller-Brown Potential U(x, y) and Gradients of V')
plt.xlabel('x')
plt.ylabel('y')
plt.legend()
plt.show()

# Plot the Muller-Brown potential and the generated samples
plt.figure(figsize=(8, 6))

# Plot the Muller-Brown potential as a contour map
plt.contourf(x_grid, y_grid, U_vals, levels=20, cmap='viridis')
plt.colorbar(label='Muller-Brown Potential U(x, y)')

# Overlay the generated samples
plt.scatter(samples[0,:,0], samples[0,:,1], color='red', marker='o', label=r'Samples $x_t$', s=0.8)
plt.quiver(samples[0,:,0], samples[0,:,1], -grad_U[0,:,0], -grad_U[0,:,1], color='orange', scale=7000, label='Gradients')
plt.title(r'Muller-Brown Potential U(x, y) and Gradients of U')
plt.xlabel('x')
plt.ylabel('y')
plt.legend()
plt.show()

#Train C Network

In [ ]:
import os

image_folder = '/content/sample_data'  # Path to the folder containing images

for filename in os.listdir(image_folder):
    if filename.endswith(".png"):
        file_path = os.path.join(image_folder, filename)
        os.remove(file_path)
        #print(f"Deleted: {filename}")

print("All .png images deleted from the folder.")

In [ ]:
epochs = 1001
lr = 0.001
nn_theta = NN_theta()
optimizer = torch.optim.Adam(nn_theta.parameters(), lr=lr)

for epoch in range(epochs):
    optimizer.zero_grad()
    t = torch.rand(1, 1)  # Sample a random t
    N = 20  # Number of samples
    samples, grad_V, grad_U = generate_samples_and_gradients_V_U(t, N, A, B, L, nn_theta)
    loss = torch.mean((grad_V - grad_U) ** 2)  # MSE loss
    loss.backward()
    optimizer.step()

    if epoch % 50 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item()}")


        #Plot potential U
        A = np.array([[-0.55828035, 1.44169]])
        B = np.array([[0.62361133, 0.02804632]])

        # Evaluate c_theta(t) over a range of t
        t_vals = torch.linspace(0, 1, 80)  # t from 0 to 1
        c_t_vals = c_theta(t_vals, A, B, nn_theta).detach().numpy()

        x, y = np.linspace(-1.5, 1.0, 100), np.linspace(-0.5, 1.7, 100)
        x, y = np.meshgrid(x, y)
        xs = torch.tensor(np.stack([x, y], -1).reshape(-1, 2), dtype=torch.float32)
        z = U(xs).reshape([100, 100])
        z = z.detach().numpy()
        plt.contourf(x, y, z, 50)
        plt.scatter(A[0, 0], A[0, 1])
        plt.scatter(B[0, 0], B[0, 1])
        plt.scatter(samples[...,0].detach().numpy(), samples[...,1].detach().numpy(), s = 0.8, color = 'skyblue')
        plt.plot(c_t_vals[:, 0], c_t_vals[:, 1])
        plt.xlim(-1.5, 1.0)
        plt.ylim(-0.5, 1.7)
        plt.savefig(f'/content/sample_data/image_{epoch}.png')  # Save with epoch number in filename
        plt.show()
        plt.close()

In [ ]:
# After the training loop is finished:
image_folder = '/content/sample_data'  # Path to the folder containing images
video_name = 'training_video.mp4'  # Output video filename

images = [img for img in os.listdir(image_folder) if img.endswith(".png")]
images.sort(key=lambda x: int(x[6:-4])) # Sort by epoch number

frame = cv2.imread(os.path.join(image_folder, images[0]))
height, width, layers = frame.shape

video = cv2.VideoWriter(video_name, cv2.VideoWriter_fourcc(*'mp4v'), 5, (width, height))  # Adjust fps as needed

for image in images:
    video.write(cv2.imread(os.path.join(image_folder, image)))

cv2.destroyAllWindows()
video.release()

#With L as a Neural Network

In [ ]:
class L_Network(nn.Module):
    def __init__(self):
        super(L_Network, self).__init__()

        # Define the layers that will output L_11, L_21, and L_22
        self.fc1 = nn.Linear(1, 16)  # Hidden layer
        self.fc2 = nn.Linear(16, 16)  # Hidden layer
        self.L_out = nn.Linear(16, 3)  # Output layer with 3 outputs (L_11, L_21, L_22)

        #Initialize linear layers with normal mean =0 and std. dev = 0.001
        nn.init.normal_(self.fc1.weight, mean=0.0, std=0.0001)
        nn.init.normal_(self.fc2.weight, mean=0.0, std=0.0001)
        nn.init.normal_(self.L_out.weight, mean=0.0, std=0.0001)

        # Initialize biases with zeros
        nn.init.zeros_(self.fc1.bias)
        nn.init.zeros_(self.fc2.bias)
        nn.init.zeros_(self.L_out.bias)

    def forward(self, t):
        # Pass t (the scalar input) through the network
        x = torch.relu(self.fc1(t))
        x = torch.relu(self.fc2(x))

        # Output the values for L_11, L_21, and L_22
        L_params = self.L_out(x)

        # Constrain L_11 and L_22 to be positive using softplus or ReLU
        L_11 = F.softplus(L_params[:, 0])  # Ensure L_11 > 0
        L_21 = L_params[:, 1]  # L_21 can be any real number
        L_22 = F.softplus(L_params[:, 2])  # Ensure L_22 > 0

        # Construct the lower triangular matrix L
        L = torch.stack([torch.stack([L_11, torch.zeros_like(L_11)], dim=-1),
                         torch.stack([L_21, L_22], dim=-1)], dim=-2)

        return L

In [ ]:
L_net = L_Network()

# Example input t (e.g., from a time series or any other input)
t = torch.tensor([[0.005]], dtype=torch.float32)

# Get the lower triangular matrix L for the given t
L_matrix = L_net(t)

# Print the resulting L matrix
print(L_matrix)

#Train L Network

In [ ]:
epochs = 10000
lr = 0.001
#nn_theta = NN_theta()
L_net = L_Network()
#optimizer = torch.optim.Adam(list(nn_theta.parameters()) + list(L_net.parameters()), lr=lr)
optimizer = torch.optim.Adam(L_net.parameters(), lr=lr)

loss_history = []

for epoch in range(epochs):
    optimizer.zero_grad()
    t = torch.rand(1, 1)  # Sample a random t
    N = 20  # Number of samples
    L = L_net(t)
    samples, grad_V, grad_U = generate_samples_and_gradients_V_U(t, N, A, B, L, nn_theta)
    loss = torch.mean((grad_V - grad_U) ** 2)  # MSE loss
    loss.backward()
    optimizer.step()

    if epoch % 100 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item()}")

        loss_history.append(loss.item())

        #Plot potential U
        A = np.array([[-0.55828035, 1.44169]])
        B = np.array([[0.62361133, 0.02804632]])

        # Evaluate c_theta(t) over a range of t
        t_vals = torch.linspace(0, 1, 100)  # t from 0 to 1
        c_t_vals = c_theta(t_vals, A, B, nn_theta).detach().numpy()

        x, y = np.linspace(-1.5, 1.0, 100), np.linspace(-0.5, 1.7, 100)
        x, y = np.meshgrid(x, y)
        xs = torch.tensor(np.stack([x, y], -1).reshape(-1, 2), dtype=torch.float32)
        z = U(xs).reshape([100, 100])
        z = z.detach().numpy()
        plt.contourf(x, y, z, 50)
        plt.scatter(A[0, 0], A[0, 1])
        plt.scatter(B[0, 0], B[0, 1])
        plt.scatter(samples[...,0].detach().numpy(), samples[...,1].detach().numpy(), s = 0.8)
        plt.plot(c_t_vals[:, 0], c_t_vals[:, 1])
        plt.show()

#Generate Samples

In [ ]:
num_samples_for_each_t = 100
t_steps = 100

t = np.linspace(0, 1, t_steps)

all_samples = []

for time_point in t[1:-1]:
    t_tensor = torch.tensor([[time_point]], dtype=torch.float32)
    L = L_net(t_tensor)
    samples, _, _ = generate_samples_and_gradients_V_U(t_tensor, num_samples_for_each_t, A, B, L, nn_theta)
    samples = samples.detach().numpy()
    all_samples.append(samples)


all_samples = np.concatenate(all_samples, axis=0)
all_samples = all_samples.reshape((t_steps-2) * num_samples_for_each_t, 2)
print(all_samples.shape)

In [ ]:
#Plot potential U
A = np.array([[-0.55828035, 1.44169]])
B = np.array([[0.62361133, 0.02804632]])

# Evaluate c_theta(t) over a range of t
t_vals = torch.linspace(0, 1, 100)  # t from 0 to 1
c_t_vals = c_theta(t_vals, A, B, nn_theta).detach().numpy()

x, y = np.linspace(-1.5, 1.0, 100), np.linspace(-0.5, 1.7, 100)
x, y = np.meshgrid(x, y)
xs = torch.tensor(np.stack([x, y], -1).reshape(-1, 2), dtype=torch.float32)
z = U(xs).reshape([100, 100])
z = z.detach().numpy()
plt.contourf(x, y, z, 50)
plt.scatter(A[0, 0], A[0, 1])
plt.scatter(B[0, 0], B[0, 1])
plt.scatter(all_samples[...,0], all_samples[...,1], s = 0.8, color = 'skyblue')
#plt.plot(c_t_vals[:, 0], c_t_vals[:, 1], color = 'blue')
plt.show()

#Plot potential $V(x,y)$

In [ ]:
#Create folder with path /content/sample_data/potential_V
import os
if not os.path.exists('/content/sample_data/potential_V'):
    os.makedirs('/content/sample_data/potential_V')

In [ ]:
t_evals = torch.linspace(0, 1, 100)

for t in t_evals[1:-1]:

    t = torch.tensor([[t]], dtype=torch.float32)
    c_t = c_theta(t, A, B, nn_theta)
    c_t = c_t.detach().numpy()

    L = L_net(t)
    L_scale = L_scaled(t, L)
    L_scale = L_scale.detach().numpy()

    x_grid, y_grid = np.meshgrid(np.linspace(-1.5, 1.0, 100), np.linspace(-0.5, 1.7, 100))
    V_grid = np.zeros_like(x_grid)

    # Calculate the potential at each grid point
    for i in range(x_grid.shape[0]):
      for j in range(x_grid.shape[1]):
        V_grid[i, j] = V(x_grid[i, j], y_grid[i, j], L_scale[0], c_t[0])

    #Plot potential U
    A = np.array([[-0.55828035, 1.44169]])
    B = np.array([[0.62361133, 0.02804632]])

    # Evaluate c_theta(t) over a range of t
    t_vals_ = torch.linspace(0, 1, 100)  # t from 0 to 1
    c_t_vals_ = c_theta(t_vals_, A, B, nn_theta).detach().numpy()

    # Plot the potential and the samples
    plt.figure(figsize=(8, 6))
    #add the max limit for the contour plot
    plt.contourf(x_grid, y_grid, V_grid, levels=30, cmap='Blues')
    plt.colorbar(label='Potential V(x, y)')
    plt.scatter(A[0, 0], A[0, 1])
    plt.scatter(B[0, 0], B[0, 1])
    plt.plot(c_t_vals_[:, 0], c_t_vals_[:, 1])
    plt.title('Potential V(x, y) and Random Samples')
    plt.xlabel('x')
    plt.ylabel('y')
    plt.savefig(f'/content/sample_data/potential_V/image_{t}.png')  # Save with epoch number in filename
    plt.show()
    plt.close()

In [ ]:
import re

def sort_key_function(filename):
    match = re.search(r"\[\[(.*?)\]\]", filename)  # Extract number using regex
    if match:
        return float(match.group(1))  # Convert to float for sorting
    else:
        return float('inf')

In [ ]:
# After the training loop is finished:
image_folder = '/content/sample_data/potential_V'  # Path to the folder containing images
video_name = 'potential_V_video.mp4'  # Output video filename

images = [img for img in os.listdir(image_folder) if img.endswith(".png")]
images.sort(key=sort_key_function) # Sort by epoch number

frame = cv2.imread(os.path.join(image_folder, images[0]))
height, width, layers = frame.shape

video = cv2.VideoWriter(video_name, cv2.VideoWriter_fourcc(*'mp4v'), 10, (width, height))  # Adjust fps as needed

for image in images:
    video.write(cv2.imread(os.path.join(image_folder, image)))

cv2.destroyAllWindows()
video.release()

#MD on U(x,y)

In [ ]:
# Langevin dynamics parameters
gamma = 1.0      # Friction coefficient
k_B_T = 10.0      # Thermal energy (k_B * Temperature)
dt = 0.0001        # Time step
n_steps = 2000000  # Number of steps

# Initialize position
position = torch.tensor([-0.55828035, 1.44169], requires_grad=False)  # Initial position as a tensor
positions = [position.numpy()]  # List to store trajectory for plotting

# Langevin dynamics loop
for i in range(n_steps):
    # Compute the gradient of the potential at the current position
    grad = grad_U_analytical(position.unsqueeze(0))[0]  # Get gradient as a tensor

    # Update position with drift and stochastic terms
    noise = torch.sqrt(torch.tensor(2 * k_B_T * dt / gamma)) * torch.randn(2)  # Noise term
    position = position - grad * dt / gamma + noise  # Langevin update

    d = (position[0] - B[0,0])**2 + (position[1] - B[0,1])**2
    if d < 0.0001:
        print('Reached B after ', i, ' steps. The time taken for simulation is', i*dt)
        break

    # Store position for plotting
    positions.append(position.numpy())

# Convert trajectory to numpy array for easy plotting
positions = np.array(positions)

# Plot the trajectory
plt.figure(figsize=(8, 6))
x, y = np.linspace(-1.5, 1.0, 100), np.linspace(-0.5, 1.7, 100)
x, y = np.meshgrid(x, y)
xs = torch.tensor(np.stack([x, y], -1).reshape(-1, 2), dtype=torch.float32)
z = U(xs).reshape([100, 100])
z = z.detach().numpy()
plt.contourf(x, y, z, 50)
plt.plot(positions[:, 0], positions[:, 1], label='Trajectory', alpha=0.8)
plt.xlabel('x')
plt.ylabel('y')
plt.xlim(-1.5, 1.0)
plt.ylim(-0.5, 1.7)
plt.scatter(A[0, 0], A[0, 1], color='red', label='A')
plt.scatter(B[0, 0], B[0, 1], color='orange', label='B')
plt.title(f'LD on Muller-Brown Potential -- Time steps taken from A to B = {i+1}')
plt.legend()
plt.show()

#MD on V(x,y)

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt

# Function to compute the potential at a given point (x, y) and time t
def V(x, y, L, c):
    diff = np.array([x - c[0], y - c[1]])
    return 0.5 * diff.T @ np.linalg.inv(L @ L.T) @ diff

# Function to compute the gradient of V with respect to x and y
def grad_V(x, y, L, c):
    diff = np.array([x - c[0], y - c[1]])
    grad = np.linalg.inv(L @ L.T) @ diff
    return grad

# Langevin dynamics parameters
gamma = 1.0         # Friction coefficient
k_B_T = 10.0         # Thermal energy (k_B * Temperature)
dt = 0.0001         # Time step for Langevin dynamics
n_steps = 1         # Number of steps for each time step in t
n_paths = 3         # Number of paths to generate

# Time-varying potential parameters
t_evals = torch.linspace(0, 1, 200)

# Generate and plot multiple Langevin dynamics paths
plt.figure(figsize=(8, 6))

for path in range(n_paths):
    # Initialize position for each path
    position = np.array([-0.55828035, 1.44169])  # Initial position
    trajectory = [position]  # List to store trajectory for the current path

    # Perform Langevin dynamics over each time slice in t_evals
    for t in t_evals[1:-1]:
        # Update parameters for time-varying potential
        t_tensor = torch.tensor([[t]], dtype=torch.float32)
        c_t = c_theta(t_tensor, A, B, nn_theta).detach().numpy()[0]
        L = L_net(t_tensor)
        L_scale = L_scaled(t_tensor, L).detach().numpy()[0]

        # Run Langevin dynamics for multiple steps in the current time slice
        for _ in range(n_steps):
            # Compute gradient of V at the current position and time
            grad = grad_V(position[0], position[1], L_scale, c_t)

            # Langevin update with deterministic and stochastic terms
            noise = np.sqrt(2 * k_B_T * dt / gamma) * np.random.randn(2)
            position = position - grad * dt / gamma + noise

            # Store position for plotting
            trajectory.append(position)

    # Convert trajectory to numpy array and plot the path
    trajectory = np.array(trajectory)
    plt.plot(trajectory[:, 0], trajectory[:, 1], label=f'Path {path + 1}')

# Plot the potential as a contour plot
x, y = np.linspace(-1.5, 1.0, 100), np.linspace(-0.5, 1.7, 100)
x, y = np.meshgrid(x, y)
xs = torch.tensor(np.stack([x, y], -1).reshape(-1, 2), dtype=torch.float32)
z = U(xs).reshape([100, 100])
z = z.detach().numpy()
#plt.contourf(x, y, z, 50, cmap="Blues", alpha=0.5)
plt.contourf(x, y, z, 50)
# Plot start and end points and labels
plt.xlabel('x')
plt.ylabel('y')
plt.title('Langevin Dynamics Trajectories on Time-Varying Potential V(x, y, t)')
plt.show()

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import os

# Create directory for saving images
output_dir = "/content/sample_data/langevin_trajectories"
os.makedirs(output_dir, exist_ok=True)

# Function to compute the potential at a given point (x, y) and time t
def V(x, y, L, c):
    diff = np.array([x - c[0], y - c[1]])
    return 0.5 * diff.T @ np.linalg.inv(L @ L.T) @ diff

# Function to compute the gradient of V with respect to x and y
def grad_V(positions, L, c):
    # Calculate gradient in parallel for multiple positions
    diff = positions - c
    grad = np.einsum('ij,jk->ik', diff, np.linalg.inv(L @ L.T))
    return grad

# Langevin dynamics parameters
gamma = 1.0         # Friction coefficient
k_B_T = 10.0         # Thermal energy (k_B * Temperature)
dt = 0.0001         # Time step for Langevin dynamics
n_steps = 1         # Number of steps for each time step in t
n_paths = 3         # Number of paths to generate

# Time-varying potential parameters
t_evals = torch.linspace(0, 1, 200)

# Initial positions for each path
positions = np.tile(np.array([-0.55828035, 1.44169]), (n_paths, 1))
trajectories = [positions.copy()]  # Initialize trajectories list

# Prepare grid for potential contour plot (for every saved frame)
x, y = np.linspace(-1.5, 1.0, 100), np.linspace(-0.5, 1.7, 100)
x, y = np.meshgrid(x, y)
xs = torch.tensor(np.stack([x, y], -1).reshape(-1, 2), dtype=torch.float32)
z = U(xs).reshape([100, 100]).detach().numpy()

# Generate and plot Langevin dynamics paths, saving each time step
for t_idx, t in enumerate(t_evals[1:-1]):
    # Update parameters for time-varying potential
    t_tensor = torch.tensor([[t]], dtype=torch.float32)
    c_t = c_theta(t_tensor, A, B, nn_theta).detach().numpy()[0]
    L = L_net(t_tensor)
    L_scale = L_scaled(t_tensor, L).detach().numpy()[0]

    # Compute gradient for all paths in parallel
    grad = grad_V(positions, L_scale, c_t)

    # Langevin update with deterministic and stochastic terms
    noise = np.sqrt(2 * k_B_T * dt / gamma) * np.random.randn(n_paths, 2)
    positions = positions - grad * dt / gamma + noise

    # Store updated positions in trajectories list
    trajectories.append(positions.copy())

    # Plot the potential and paths at this time step
    plt.figure(figsize=(8, 6))
    plt.contourf(x, y, z, 50)
    for path_idx in range(n_paths):
        # Extract path history up to the current time step
        path_history = np.array([trajectory[path_idx] for trajectory in trajectories])
        plt.plot(path_history[:, 0], path_history[:, 1], label=f'Path {path_idx + 1}')

    # Plot configuration for each frame
    plt.xlabel('x')
    plt.ylabel('y')
    plt.title(f'Langevin Dynamics Trajectories at Time Step {t_idx + 1}')
    plt.legend()
    # Save each frame as an image file
    plt.savefig(f"{output_dir}/frame_{t_idx + 1:03d}.png")
    plt.show()
    plt.close()

In [ ]:
def sort_key_function(filename):
    # Extract the number from the filename "frame_xxx.png"
    return int(filename.split('_')[1].split('.')[0])

# After the training loop is finished:
image_folder = '/content/sample_data/langevin_trajectories'  # Path to the folder containing images
video_name = 'MD_V_video.mp4'  # Output video filename

# Get list of image files and sort by frame number
images = [img for img in os.listdir(image_folder) if img.endswith(".png")]
images.sort(key=sort_key_function)  # Sort by frame number using the defined function

# Read the first frame to get video dimensions
frame = cv2.imread(os.path.join(image_folder, images[0]))
height, width, layers = frame.shape

# Create the video writer object
video = cv2.VideoWriter(video_name, cv2.VideoWriter_fourcc(*'mp4v'), 10, (width, height))  # Adjust fps as needed

# Write each image to the video
for image in images:
    video.write(cv2.imread(os.path.join(image_folder, image)))

# Release the video writer and close windows
cv2.destroyAllWindows()
video.release()

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import os

# Create directory for saving images
output_dir = "/content/sample_data/langevin_trajectories"
os.makedirs(output_dir, exist_ok=True)

# Function to compute the potential V at a given point (x, y) and time t
def V(x, y, L, c):
    diff = np.array([x - c[0], y - c[1]])
    return 0.5 * diff.T @ np.linalg.inv(L @ L.T) @ diff

# Function to compute the gradient of V with respect to x and y
def grad_V(positions, L, c):
    # Calculate gradient in parallel for multiple positions
    diff = positions - c
    grad = np.einsum('ij,jk->ik', diff, np.linalg.inv(L @ L.T))
    return grad

# Langevin dynamics parameters
gamma = 1.0         # Friction coefficient
k_B_T = 10.0        # Thermal energy (k_B * Temperature)
dt = 0.0001         # Time step for Langevin dynamics
n_steps = 1         # Number of steps for each time step in t
n_paths = 3         # Number of paths to generate

# Time-varying potential parameters
t_evals = torch.linspace(0, 1, 200)

# Initial positions for each path
positions = np.tile(np.array([-0.55828035, 1.44169]), (n_paths, 1))
trajectories = [positions.copy()]  # Initialize trajectories list

# Prepare grid for the fixed potential U
x, y = np.linspace(-1.5, 1.0, 100), np.linspace(-0.5, 1.7, 100)
x, y = np.meshgrid(x, y)
xs = torch.tensor(np.stack([x, y], -1).reshape(-1, 2), dtype=torch.float32)
z_U = U(xs).reshape([100, 100]).detach().numpy()  # Fixed potential U

A = np.array([[-0.55828035, 1.44169]])
B = np.array([[0.62361133, 0.02804632]])

# Evaluate c_theta(t) over a range of t
t_vals_ = torch.linspace(0, 1, 100)  # t from 0 to 1
c_t_vals_ = c_theta(t_vals_, A, B, nn_theta).detach().numpy()

# Generate and plot Langevin dynamics paths, saving each time step
for t_idx, t in enumerate(t_evals[1:-1]):
    # Update parameters for time-varying potential V
    t_tensor = torch.tensor([[t]], dtype=torch.float32)
    c_t = c_theta(t_tensor, A, B, nn_theta).detach().numpy()[0]
    L = L_net(t_tensor)
    L_scale = L_scaled(t_tensor, L).detach().numpy()[0]

    # Compute gradient for all paths in parallel
    grad = grad_V(positions, L_scale, c_t)

    # Langevin update with deterministic and stochastic terms
    noise = np.sqrt(2 * k_B_T * dt / gamma) * np.random.randn(n_paths, 2)
    positions = positions - grad * dt / gamma + noise

    # Store updated positions in trajectories list
    trajectories.append(positions.copy())

    # Create a figure with two subplots side by side
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))
    fig.suptitle(f'Time Step {t_idx + 1}')
    # Left subplot: Plot the fixed potential U with Langevin paths
    ax1.contourf(x, y, z_U, 50)
    for path_idx in range(n_paths):
        # Extract path history up to the current time step
        path_history = np.array([trajectory[path_idx] for trajectory in trajectories])
        ax1.plot(path_history[:, 0], path_history[:, 1], label=f'Path {path_idx + 1}')
    ax1.set_title(f'LD Paths with Muller-Brown Background')
    ax1.set_xlabel('x')
    ax1.set_ylabel('y')
    ax1.legend()

    # Right subplot: Plot the time-varying potential V at current time step
    #Plot potential U

    z_V = np.array([[V(x_val, y_val, L_scale, c_t) for x_val, y_val in zip(x_row, y_row)] for x_row, y_row in zip(x, y)])
    ax2.contourf(x, y, z_V, levels=50, cmap='viridis')
    ax2.scatter(positions[:, 0], positions[:, 1], color='red', label='Current Position')
    ax2.scatter(A[0, 0], A[0, 1])
    ax2.scatter(B[0, 0], B[0, 1])
    ax2.plot(c_t_vals_[:, 0], c_t_vals_[:, 1])
    ax2.set_title('Time-Varying Potential V(x, y, t) with Current Positions')
    ax2.set_xlabel('x')
    ax2.set_ylabel('y')
    ax2.legend()

    # Save each frame as an image file
    plt.savefig(f"{output_dir}/frame_{t_idx + 1:03d}.png")
    plt.show()
    plt.close()

In [ ]:
def sort_key_function(filename):
    # Extract the number from the filename "frame_xxx.png"
    return int(filename.split('_')[1].split('.')[0])

# After the training loop is finished:
image_folder = '/content/sample_data/langevin_trajectories'  # Path to the folder containing images
video_name = 'MD_V_video.mp4'  # Output video filename

# Get list of image files and sort by frame number
images = [img for img in os.listdir(image_folder) if img.endswith(".png")]
images.sort(key=sort_key_function)  # Sort by frame number using the defined function

# Read the first frame to get video dimensions
frame = cv2.imread(os.path.join(image_folder, images[0]))
height, width, layers = frame.shape

# Create the video writer object
video = cv2.VideoWriter(video_name, cv2.VideoWriter_fourcc(*'mp4v'), 10, (width, height))  # Adjust fps as needed

# Write each image to the video
for image in images:
    video.write(cv2.imread(os.path.join(image_folder, image)))

# Release the video writer and close windows
cv2.destroyAllWindows()
video.release()